# Milestone 08: 參考解答 | Reference Solution

> **專案名稱**: M07 Todo App 模組化重構  
> **原始專案**: Milestone 07 - Todo App  
> **重構範例**: 展示如何將單一檔案重構為模組化結構  
> **注意**: 這是一個參考範例，你應該選擇自己的專案進行重構

## 📋 重構概覽 | Refactoring Overview

### 原始結構

```
milestone07-todo-app/
└── solution.ipynb    # 單一檔案，約 350 行
```

### 重構後結構

```
todo_app_refactored/
├── README.md              # 專案說明文件
├── requirements.txt       # 依賴清單（空檔案）
├── .gitignore            # Git 忽略檔案
├── src/                  # 原始碼目錄
│   ├── __init__.py       # 套件初始化
│   ├── main.py           # 主程式入口
│   ├── task.py           # Task 類別
│   ├── task_manager.py   # TaskManager 類別
│   ├── storage.py        # JSON 檔案讀寫
│   ├── ui.py             # 使用者界面
│   ├── utils.py          # 工具函式（日期驗證等）
│   └── exceptions.py     # 自訂例外
└── data/
    └── tasks.json        # 資料檔案
```

### 模組職責劃分

| 模組 | 職責 | 主要內容 |
|:-----|:-----|:---------|
| `task.py` | 任務資料結構 | Task 類別定義 |
| `task_manager.py` | 任務管理邏輯 | TaskManager 類別，CRUD 操作 |
| `storage.py` | 資料持久化 | JSON 檔案讀寫函式 |
| `ui.py` | 使用者互動 | 選單顯示、輸入輸出 |
| `utils.py` | 工具函式 | 日期驗證、格式化、常數 |
| `exceptions.py` | 錯誤處理 | 自訂例外類別 |
| `main.py` | 主程式 | 整合所有模組，控制流程 |

---

## 模組 1: task.py - 任務類別

In [ ]:
# src/task.py
"""任務資料結構模組

定義 Task 類別，封裝任務的所有屬性與行為。
"""

from datetime import datetime
from typing import Dict, Any, Optional


class Task:
    """任務類別
    
    封裝單一任務的所有資訊與操作。
    
    Attributes:
        task_id: 任務唯一識別碼
        title: 任務標題
        description: 任務描述
        priority: 優先度 (1-5)
        due_date: 截止日期 (YYYY-MM-DD 格式字串)
        completed: 是否完成
        created_at: 建立時間
    
    Examples:
        >>> task = Task(
        ...     task_id="T001",
        ...     title="完成作業",
        ...     priority=3
        ... )
        >>> task.mark_complete()
        >>> print(task.completed)
        True
    """
    
    def __init__(
        self,
        task_id: str,
        title: str,
        description: str = "",
        priority: int = 1,
        due_date: Optional[str] = None,
        completed: bool = False,
        created_at: Optional[str] = None
    ) -> None:
        """初始化任務物件
        
        Args:
            task_id: 任務唯一識別碼
            title: 任務標題
            description: 任務描述，預設為空字串
            priority: 優先度 (1-5)，預設為 1
            due_date: 截止日期 (YYYY-MM-DD)，可選
            completed: 是否完成，預設為 False
            created_at: 建立時間，若未提供則使用當前時間
        """
        self.task_id: str = task_id
        self.title: str = title
        self.description: str = description
        self.priority: int = priority
        self.due_date: Optional[str] = due_date
        self.completed: bool = completed
        self.created_at: str = created_at or datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    def mark_complete(self) -> None:
        """標記任務為已完成"""
        self.completed = True
    
    def mark_incomplete(self) -> None:
        """標記任務為未完成"""
        self.completed = False
    
    def to_dict(self) -> Dict[str, Any]:
        """轉換為字典格式
        
        用於 JSON 序列化。
        
        Returns:
            包含任務所有資訊的字典
        """
        return {
            'task_id': self.task_id,
            'title': self.title,
            'description': self.description,
            'priority': self.priority,
            'due_date': self.due_date,
            'completed': self.completed,
            'created_at': self.created_at
        }
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Task':
        """從字典建立 Task 物件
        
        用於 JSON 反序列化。
        
        Args:
            data: 包含任務資訊的字典
        
        Returns:
            Task 物件實例
        """
        return cls(
            task_id=data['task_id'],
            title=data['title'],
            description=data.get('description', ''),
            priority=data.get('priority', 1),
            due_date=data.get('due_date'),
            completed=data.get('completed', False),
            created_at=data.get('created_at')
        )
    
    def __str__(self) -> str:
        """字串表示
        
        Returns:
            任務的字串描述
        """
        status = "✓" if self.completed else "☐"
        return f"[{status}] {self.title} (優先度: {self.priority})"
    
    def __repr__(self) -> str:
        """開發者友善的字串表示
        
        Returns:
            完整的物件資訊
        """
        return f"Task(id={self.task_id}, title='{self.title}', completed={self.completed})"

---

## 模組 2: task_manager.py - 任務管理器

In [ ]:
# src/task_manager.py
"""任務管理器模組

定義 TaskManager 類別，負責管理所有任務的 CRUD 操作。
"""

from typing import List, Optional
from .task import Task
from .exceptions import TaskNotFoundError, DuplicateTaskError


class TaskManager:
    """任務管理器類別
    
    負責管理任務清單的所有操作，包含新增、查詢、修改、刪除。
    
    Attributes:
        tasks: 任務列表
    
    Examples:
        >>> manager = TaskManager()
        >>> task = Task("T001", "完成作業")
        >>> manager.add_task(task)
        >>> len(manager.get_all_tasks())
        1
    """
    
    def __init__(self) -> None:
        """初始化任務管理器"""
        self.tasks: List[Task] = []
    
    def add_task(self, task: Task) -> None:
        """新增任務
        
        Args:
            task: 要新增的 Task 物件
        
        Raises:
            DuplicateTaskError: 當任務 ID 已存在時
        """
        if self.get_task_by_id(task.task_id) is not None:
            raise DuplicateTaskError(task.task_id)
        self.tasks.append(task)
    
    def remove_task(self, task_id: str) -> bool:
        """移除任務
        
        Args:
            task_id: 要移除的任務 ID
        
        Returns:
            是否成功移除
        
        Raises:
            TaskNotFoundError: 當找不到指定任務時
        """
        task = self.get_task_by_id(task_id)
        if task is None:
            raise TaskNotFoundError(task_id)
        
        self.tasks.remove(task)
        return True
    
    def get_task_by_id(self, task_id: str) -> Optional[Task]:
        """根據 ID 取得任務
        
        Args:
            task_id: 任務 ID
        
        Returns:
            找到的 Task 物件，若未找到則回傳 None
        """
        for task in self.tasks:
            if task.task_id == task_id:
                return task
        return None
    
    def get_all_tasks(self) -> List[Task]:
        """取得所有任務
        
        Returns:
            任務列表
        """
        return self.tasks
    
    def get_pending_tasks(self) -> List[Task]:
        """取得未完成的任務
        
        Returns:
            未完成任務列表
        """
        return [t for t in self.tasks if not t.completed]
    
    def get_completed_tasks(self) -> List[Task]:
        """取得已完成的任務
        
        Returns:
            已完成任務列表
        """
        return [t for t in self.tasks if t.completed]
    
    def get_tasks_by_priority(self, priority: int) -> List[Task]:
        """根據優先度篩選任務
        
        Args:
            priority: 優先度 (1-5)
        
        Returns:
            符合條件的任務列表
        """
        return [t for t in self.tasks if t.priority == priority]
    
    def get_sorted_tasks(self, by: str = 'priority', reverse: bool = True) -> List[Task]:
        """取得排序後的任務列表
        
        Args:
            by: 排序依據 ('priority', 'title', 'due_date')
            reverse: 是否降序排列
        
        Returns:
            排序後的任務列表
        """
        if by == 'priority':
            return sorted(self.tasks, key=lambda t: t.priority, reverse=reverse)
        elif by == 'title':
            return sorted(self.tasks, key=lambda t: t.title, reverse=reverse)
        elif by == 'due_date':
            # 將 None 視為最晚
            return sorted(
                self.tasks,
                key=lambda t: t.due_date if t.due_date else '9999-12-31',
                reverse=reverse
            )
        return self.tasks
    
    def mark_task_complete(self, task_id: str) -> None:
        """標記任務為完成
        
        Args:
            task_id: 任務 ID
        
        Raises:
            TaskNotFoundError: 當找不到指定任務時
        """
        task = self.get_task_by_id(task_id)
        if task is None:
            raise TaskNotFoundError(task_id)
        task.mark_complete()
    
    def get_task_count(self) -> int:
        """取得任務總數
        
        Returns:
            任務數量
        """
        return len(self.tasks)
    
    def clear_all_tasks(self) -> None:
        """清空所有任務"""
        self.tasks = []

---

## 模組 3: storage.py - 資料儲存

In [ ]:
# src/storage.py
"""資料儲存模組

負責 JSON 檔案的讀取與寫入操作。
"""

import json
from pathlib import Path
from typing import List, Dict, Any
from .task import Task
from .task_manager import TaskManager


def load_tasks(file_path: Path) -> List[Task]:
    """從 JSON 檔案載入任務
    
    Args:
        file_path: JSON 檔案路徑
    
    Returns:
        Task 物件列表
    
    Raises:
        json.JSONDecodeError: 當 JSON 格式錯誤時
    
    Examples:
        >>> path = Path("data/tasks.json")
        >>> tasks = load_tasks(path)
        >>> len(tasks)
        5
    """
    if not file_path.exists():
        return []
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 將字典列表轉換為 Task 物件列表
        return [Task.from_dict(task_data) for task_data in data]
    
    except json.JSONDecodeError as e:
        print(f"JSON 格式錯誤: {e}")
        return []
    except Exception as e:
        print(f"讀取檔案時發生錯誤: {e}")
        return []


def save_tasks(tasks: List[Task], file_path: Path) -> bool:
    """儲存任務到 JSON 檔案
    
    Args:
        tasks: Task 物件列表
        file_path: JSON 檔案路徑
    
    Returns:
        是否成功儲存
    
    Examples:
        >>> tasks = [Task("T001", "完成作業")]
        >>> path = Path("data/tasks.json")
        >>> save_tasks(tasks, path)
        True
    """
    try:
        # 確保目錄存在
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        # 將 Task 物件列表轉換為字典列表
        data = [task.to_dict() for task in tasks]
        
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        return True
    
    except Exception as e:
        print(f"儲存檔案時發生錯誤: {e}")
        return False


def load_manager(file_path: Path) -> TaskManager:
    """從檔案載入任務並建立 TaskManager
    
    Args:
        file_path: JSON 檔案路徑
    
    Returns:
        包含載入任務的 TaskManager 物件
    """
    manager = TaskManager()
    tasks = load_tasks(file_path)
    
    for task in tasks:
        try:
            manager.add_task(task)
        except Exception as e:
            print(f"載入任務 {task.task_id} 時發生錯誤: {e}")
    
    return manager


def save_manager(manager: TaskManager, file_path: Path) -> bool:
    """儲存 TaskManager 的所有任務
    
    Args:
        manager: TaskManager 物件
        file_path: JSON 檔案路徑
    
    Returns:
        是否成功儲存
    """
    return save_tasks(manager.get_all_tasks(), file_path)

---

## 模組 4: utils.py - 工具函式

In [ ]:
# src/utils.py
"""工具函式模組

提供日期驗證、格式化等輔助功能。
"""

from datetime import datetime
from typing import Optional

# ========== 常數定義 ==========

DATE_FORMAT = "%Y-%m-%d"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"
MIN_PRIORITY = 1
MAX_PRIORITY = 5
DEFAULT_PRIORITY = 1

# 優先度描述
PRIORITY_LABELS = {
    1: "低",
    2: "中低",
    3: "中",
    4: "中高",
    5: "高"
}


# ========== 日期處理函式 ==========

def validate_date(date_str: str) -> bool:
    """驗證日期格式是否正確
    
    Args:
        date_str: 日期字串 (YYYY-MM-DD)
    
    Returns:
        是否為有效日期格式
    
    Examples:
        >>> validate_date("2025-10-08")
        True
        >>> validate_date("2025/10/08")
        False
        >>> validate_date("invalid")
        False
    """
    try:
        datetime.strptime(date_str, DATE_FORMAT)
        return True
    except ValueError:
        return False


def parse_date(date_str: str) -> Optional[datetime]:
    """解析日期字串
    
    Args:
        date_str: 日期字串 (YYYY-MM-DD)
    
    Returns:
        datetime 物件，若格式錯誤則回傳 None
    """
    try:
        return datetime.strptime(date_str, DATE_FORMAT)
    except ValueError:
        return None


def format_date(date: datetime) -> str:
    """格式化日期
    
    Args:
        date: datetime 物件
    
    Returns:
        格式化的日期字串 (YYYY-MM-DD)
    """
    return date.strftime(DATE_FORMAT)


def get_current_date() -> str:
    """取得當前日期
    
    Returns:
        當前日期字串 (YYYY-MM-DD)
    """
    return datetime.now().strftime(DATE_FORMAT)


def is_overdue(due_date: str) -> bool:
    """檢查任務是否逾期
    
    Args:
        due_date: 截止日期字串 (YYYY-MM-DD)
    
    Returns:
        是否逾期
    """
    if not due_date:
        return False
    
    try:
        due = datetime.strptime(due_date, DATE_FORMAT)
        return due < datetime.now()
    except ValueError:
        return False


# ========== 優先度處理函式 ==========

def validate_priority(priority: int) -> bool:
    """驗證優先度是否有效
    
    Args:
        priority: 優先度 (1-5)
    
    Returns:
        是否有效
    
    Examples:
        >>> validate_priority(3)
        True
        >>> validate_priority(6)
        False
    """
    return MIN_PRIORITY <= priority <= MAX_PRIORITY


def get_priority_label(priority: int) -> str:
    """取得優先度的中文標籤
    
    Args:
        priority: 優先度 (1-5)
    
    Returns:
        優先度標籤
    
    Examples:
        >>> get_priority_label(5)
        '高'
        >>> get_priority_label(1)
        '低'
    """
    return PRIORITY_LABELS.get(priority, "未知")


# ========== 其他工具函式 ==========

def generate_task_id(existing_ids: set) -> str:
    """產生唯一的任務 ID
    
    Args:
        existing_ids: 現有的 ID 集合
    
    Returns:
        新的唯一 ID (格式: T001, T002, ...)
    """
    counter = 1
    while True:
        new_id = f"T{counter:03d}"
        if new_id not in existing_ids:
            return new_id
        counter += 1


def truncate_text(text: str, max_length: int = 50) -> str:
    """截斷文字並加上省略號
    
    Args:
        text: 要截斷的文字
        max_length: 最大長度
    
    Returns:
        截斷後的文字
    
    Examples:
        >>> truncate_text("This is a very long text", 10)
        'This is...'
    """
    if len(text) <= max_length:
        return text
    return text[:max_length - 3] + "..."

---

## 模組 5: exceptions.py - 自訂例外

In [ ]:
# src/exceptions.py
"""自訂例外模組

定義 Todo App 專用的例外類別。
"""


class TodoAppError(Exception):
    """Todo App 基礎例外類別
    
    所有 Todo App 相關的例外都繼承此類別。
    """
    pass


class TaskNotFoundError(TodoAppError):
    """找不到任務時拋出的例外
    
    Examples:
        >>> raise TaskNotFoundError("T001")
        TodoAppError: 找不到任務: T001
    """
    
    def __init__(self, task_id: str) -> None:
        """初始化例外
        
        Args:
            task_id: 找不到的任務 ID
        """
        self.task_id = task_id
        super().__init__(f"找不到任務: {task_id}")


class DuplicateTaskError(TodoAppError):
    """任務 ID 重複時拋出的例外
    
    Examples:
        >>> raise DuplicateTaskError("T001")
        TodoAppError: 任務 ID 已存在: T001
    """
    
    def __init__(self, task_id: str) -> None:
        """初始化例外
        
        Args:
            task_id: 重複的任務 ID
        """
        self.task_id = task_id
        super().__init__(f"任務 ID 已存在: {task_id}")


class InvalidDateFormatError(TodoAppError):
    """日期格式錯誤時拋出的例外
    
    Examples:
        >>> raise InvalidDateFormatError("2025/10/08")
        TodoAppError: 日期格式錯誤: 2025/10/08 (應為 YYYY-MM-DD)
    """
    
    def __init__(self, date_str: str) -> None:
        """初始化例外
        
        Args:
            date_str: 錯誤的日期字串
        """
        self.date_str = date_str
        super().__init__(f"日期格式錯誤: {date_str} (應為 YYYY-MM-DD)")


class InvalidPriorityError(TodoAppError):
    """優先度無效時拋出的例外
    
    Examples:
        >>> raise InvalidPriorityError(10)
        TodoAppError: 優先度無效: 10 (應介於 1-5 之間)
    """
    
    def __init__(self, priority: int) -> None:
        """初始化例外
        
        Args:
            priority: 無效的優先度值
        """
        self.priority = priority
        super().__init__(f"優先度無效: {priority} (應介於 1-5 之間)")

---

## 模組 6: ui.py - 使用者界面

In [ ]:
# src/ui.py
"""使用者界面模組

負責所有使用者互動、選單顯示、輸入輸出。
"""

from typing import Optional
from .task import Task
from .task_manager import TaskManager
from .utils import (
    validate_date,
    validate_priority,
    get_priority_label,
    is_overdue,
    truncate_text
)


# ========== 選單顯示 ==========

def display_main_menu() -> None:
    """顯示主選單"""
    print("\n" + "=" * 50)
    print("    待辦事項管理系統 | Todo App")
    print("=" * 50)
    print("1. 新增任務")
    print("2. 顯示所有任務")
    print("3. 顯示未完成任務")
    print("4. 標記任務為完成")
    print("5. 刪除任務")
    print("6. 搜尋任務")
    print("0. 儲存並離開")
    print("=" * 50)


def get_menu_choice() -> str:
    """取得使用者的選單選擇
    
    Returns:
        使用者輸入的選項
    """
    return input("\n請選擇功能 (0-6): ").strip()


# ========== 任務顯示 ==========

def display_tasks(tasks: list[Task], title: str = "任務清單") -> None:
    """顯示任務列表
    
    Args:
        tasks: 任務列表
        title: 清單標題
    """
    if not tasks:
        print(f"\n{title}: (無任務)")
        return
    
    print(f"\n{'=' * 80}")
    print(f"  {title} (共 {len(tasks)} 項)")
    print("=" * 80)
    print(f"{'ID':<6} {'狀態':<4} {'標題':<30} {'優先度':<8} {'截止日期':<12}")
    print("-" * 80)
    
    for task in tasks:
        status = "✓" if task.completed else "☐"
        title_display = truncate_text(task.title, 28)
        priority_label = f"{task.priority} ({get_priority_label(task.priority)})"
        due_date_display = task.due_date if task.due_date else "-"
        
        # 標記逾期任務
        if not task.completed and task.due_date and is_overdue(task.due_date):
            due_date_display += " (逾期)"
        
        print(f"{task.task_id:<6} {status:<4} {title_display:<30} {priority_label:<8} {due_date_display:<12}")
    
    print("=" * 80)


def display_task_detail(task: Task) -> None:
    """顯示任務詳細資訊
    
    Args:
        task: Task 物件
    """
    print("\n" + "=" * 50)
    print("  任務詳細資訊")
    print("=" * 50)
    print(f"ID: {task.task_id}")
    print(f"標題: {task.title}")
    print(f"描述: {task.description if task.description else '(無)'}") 
    print(f"優先度: {task.priority} ({get_priority_label(task.priority)})")
    print(f"截止日期: {task.due_date if task.due_date else '(未設定)'}")
    print(f"狀態: {'已完成 ✓' if task.completed else '未完成 ☐'}")
    print(f"建立時間: {task.created_at}")
    print("=" * 50)


# ========== 輸入函式 ==========

def get_string_input(prompt: str, allow_empty: bool = False) -> Optional[str]:
    """取得字串輸入
    
    Args:
        prompt: 提示訊息
        allow_empty: 是否允許空輸入
    
    Returns:
        使用者輸入的字串，若為空且不允許則回傳 None
    """
    value = input(prompt).strip()
    
    if not value and not allow_empty:
        print("錯誤：輸入不可為空")
        return None
    
    return value if value else None


def get_integer_input(prompt: str, min_val: int, max_val: int) -> Optional[int]:
    """取得整數輸入
    
    Args:
        prompt: 提示訊息
        min_val: 最小值
        max_val: 最大值
    
    Returns:
        整數值，若無效則回傳 None
    """
    try:
        value = int(input(prompt))
        if min_val <= value <= max_val:
            return value
        else:
            print(f"錯誤：請輸入 {min_val} 到 {max_val} 之間的數字")
            return None
    except ValueError:
        print("錯誤：請輸入有效的數字")
        return None


def get_date_input(prompt: str, allow_empty: bool = True) -> Optional[str]:
    """取得日期輸入
    
    Args:
        prompt: 提示訊息
        allow_empty: 是否允許空輸入
    
    Returns:
        日期字串 (YYYY-MM-DD)，若無效則回傳 None
    """
    value = input(prompt).strip()
    
    if not value:
        return None if allow_empty else None
    
    if validate_date(value):
        return value
    else:
        print("錯誤：日期格式不正確，應為 YYYY-MM-DD")
        return None


# ========== 訊息顯示 ==========

def show_success(message: str) -> None:
    """顯示成功訊息
    
    Args:
        message: 訊息內容
    """
    print(f"\n✓ {message}")


def show_error(message: str) -> None:
    """顯示錯誤訊息
    
    Args:
        message: 訊息內容
    """
    print(f"\n✗ 錯誤: {message}")


def show_info(message: str) -> None:
    """顯示資訊訊息
    
    Args:
        message: 訊息內容
    """
    print(f"\nℹ {message}")


def confirm_action(prompt: str) -> bool:
    """請求使用者確認動作
    
    Args:
        prompt: 確認訊息
    
    Returns:
        使用者是否確認
    """
    response = input(f"{prompt} (y/n): ").strip().lower()
    return response == 'y'

---

## 模組 7: __init__.py - 套件初始化

In [ ]:
# src/__init__.py
"""Todo App 套件

一個簡單但功能完整的待辦事項管理系統。

此套件提供以下功能：
- 新增、查詢、修改、刪除任務
- 任務優先度管理
- 任務截止日期管理
- JSON 檔案持久化儲存
- 友善的命令列界面

Examples:
    基本使用：
    
    >>> from src import Task, TaskManager
    >>> manager = TaskManager()
    >>> task = Task("T001", "完成作業", priority=3)
    >>> manager.add_task(task)
    >>> len(manager.get_all_tasks())
    1
"""

__version__ = "1.0.0"
__author__ = "iSpan Python Course"
__all__ = [
    # 核心類別
    "Task",
    "TaskManager",
    # 主程式
    "main",
    # 例外
    "TodoAppError",
    "TaskNotFoundError",
    "DuplicateTaskError",
    "InvalidDateFormatError",
    "InvalidPriorityError",
]

# 匯入核心功能
from .task import Task
from .task_manager import TaskManager
from .main import main
from .exceptions import (
    TodoAppError,
    TaskNotFoundError,
    DuplicateTaskError,
    InvalidDateFormatError,
    InvalidPriorityError
)

---

## 模組 8: main.py - 主程式（節錄）

**完整版本請參考實際檔案**

In [ ]:
# src/main.py (節錄)
"""主程式模組

程式的入口點，整合所有模組功能。
"""

from pathlib import Path
from .task import Task
from .task_manager import TaskManager
from .storage import load_manager, save_manager
from .ui import (
    display_main_menu,
    get_menu_choice,
    display_tasks,
    get_string_input,
    get_integer_input,
    get_date_input,
    show_success,
    show_error,
    show_info,
    confirm_action
)
from .utils import generate_task_id, MIN_PRIORITY, MAX_PRIORITY
from .exceptions import TaskNotFoundError, DuplicateTaskError

# 資料檔案路徑
DATA_FILE = Path("data/tasks.json")


def handle_add_task(manager: TaskManager) -> None:
    """處理新增任務
    
    Args:
        manager: TaskManager 物件
    """
    print("\n=== 新增任務 ===")
    
    # 取得任務資訊
    title = get_string_input("請輸入任務標題: ")
    if not title:
        return
    
    description = get_string_input("請輸入任務描述 (可選): ", allow_empty=True)
    priority = get_integer_input(
        f"請輸入優先度 ({MIN_PRIORITY}-{MAX_PRIORITY}): ",
        MIN_PRIORITY,
        MAX_PRIORITY
    )
    if not priority:
        return
    
    due_date = get_date_input("請輸入截止日期 (YYYY-MM-DD，可選): ")
    
    # 產生唯一 ID
    existing_ids = {t.task_id for t in manager.get_all_tasks()}
    task_id = generate_task_id(existing_ids)
    
    # 建立並新增任務
    task = Task(
        task_id=task_id,
        title=title,
        description=description or "",
        priority=priority,
        due_date=due_date
    )
    
    try:
        manager.add_task(task)
        show_success(f"任務已新增 (ID: {task_id})")
    except DuplicateTaskError as e:
        show_error(str(e))


def main() -> None:
    """主程式入口"""
    print("\n歡迎使用待辦事項管理系統！")
    
    # 載入任務資料
    manager = load_manager(DATA_FILE)
    show_info(f"已載入 {manager.get_task_count()} 個任務")
    
    # 主迴圈
    while True:
        display_main_menu()
        choice = get_menu_choice()
        
        if choice == "1":
            handle_add_task(manager)
        elif choice == "2":
            tasks = manager.get_sorted_tasks(by='priority')
            display_tasks(tasks, "所有任務")
        elif choice == "3":
            tasks = manager.get_pending_tasks()
            display_tasks(tasks, "未完成任務")
        # ... 其他功能 ...
        elif choice == "0":
            if save_manager(manager, DATA_FILE):
                show_success("任務已儲存")
            print("\n感謝使用，再見！")
            break
        else:
            show_error("無效的選項")


if __name__ == "__main__":
    main()

---

## 重構總結 | Summary

### 重構成果

1. **模組化設計**
   - 將 350 行單一檔案拆解為 8 個模組
   - 每個模組職責單一且明確
   - 模組間低耦合、高內聚

2. **程式碼品質提升**
   - 完全符合 PEP 8 規範
   - 所有函式都有型別提示
   - 完整的 docstring (Google Style)
   - 移除所有 magic numbers

3. **維護性提升**
   - 清晰的專案結構
   - 詳細的文件說明
   - 自訂例外處理
   - 易於擴展與修改

4. **專業工程實務**
   - Git 版本控制
   - 虛擬環境管理
   - .gitignore 設定
   - 專案文件齊全

### 重構前後對比

| 項目 | 重構前 | 重構後 |
|:-----|:-------|:-------|
| 檔案數量 | 1 | 8+ |
| 模組化程度 | 低 | 高 |
| 型別提示 | 無 | 完整 |
| Docstring | 部分 | 完整 |
| 錯誤處理 | 基本 | 自訂例外 |
| 可測試性 | 低 | 高 |
| 可維護性 | 低 | 高 |
| 可擴展性 | 低 | 高 |

### 學習要點

1. **單一職責原則** - 每個模組只負責一個功能領域
2. **關注點分離** - UI 與邏輯完全分離
3. **型別安全** - 使用型別提示提早發現錯誤
4. **文件優先** - 詳細的 docstring 與 README
5. **錯誤處理** - 自訂例外提供清楚的錯誤訊息
6. **版本控制** - Git 記錄開發歷程

---

**這是一個完整的模組化重構範例，展示了如何將單一檔案程式重構為專業的模組化結構。**